# BL 1-5: SAXS data correction and reduction



#!/usr/bin/env python3

"""
Created on Fri Apr 21 23:41:02 2023

@author: akmaurya
"""

Global stuff

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
#from axis_plot_prop import axis_plot_prop 
import pyFAI
import shutil
import fabio
from pyFAI.gui.jupyter.calib import Calibration


import warnings
warnings.filterwarnings('ignore')


# Common functions for the project

In [ ]:
# plot setting fucntion

def axis_plot_prop(ax):
    """
    Set axis properties for plotting.

    Parameters
    ----------
    ax : matplotlib axis object
        The axis object to set properties for.
    """

    co = [
        [0, 0, 0],
        [1, 0, 0],
        [0.44, 0.00, 0.99],
        [1.00, 0.50, 0.10],
        [0.75, 0.00, 0.75],
        [0.50, 0.50, 0.50],
        [0.50, 0.57, 0.00],
        [0.64, 0.08, 0.18],
        [0.93, 0.00, 0.00]
    ]

    plt.rcParams['axes.prop_cycle'] = plt.cycler(color=co)

    ax.set_facecolor('white')
    ax.spines['top'].set_linewidth(1)
    ax.spines['right'].set_linewidth(1)
    ax.spines['bottom'].set_linewidth(1)
    ax.spines['left'].set_linewidth(1)
    ax.tick_params(axis='both', which='both', direction='in', length=4, width=1)
    ax.minorticks_on()
    ax.set_xlabel('X Label', fontsize=20, fontname='Arial')
    ax.set_ylabel('Y Label', fontsize=20, fontname='Arial')
    ax.tick_params(axis='both', labelsize=20)
    ax.legend(frameon=False)
    #ax.set_box_aspect(1)
    plt.tight_layout()
#2D plot settings function
# import matplotlib.pyplot as plt

def set_pcolor_properties():
    """
    Sets the best figure properties for pcolormap plot
    """
    plt.rcParams["figure.figsize"] = (6, 6)  # Set the figure size to 8 inches by 6 inches
    plt.rcParams["font.size"] = 20  # Set the font size to 12 points
    plt.rcParams["axes.linewidth"] = 1.5  # Set the linewidth of the axes to 1.5
    #plt.rcParams["axes.grid"] = True  # Show the grid
    #plt.rcParams["grid.alpha"] = 0.5  # Set the transparency of the grid to 0.5
    #plt.rcParams["grid.linewidth"] = 0.5  # Set the linewidth of the grid to 0.5
    plt.rcParams["image.cmap"] = "viridis"  # Set the colormap to viridis
    plt.rcParams["image.interpolation"] = "nearest"  # Set the interpolation to nearest

# create and update folder to save data
def create_update_subfolder(base_folder, common_keyword):
    common_folder_path = os.path.join(base_folder, common_keyword)
    
    if os.path.exists(base_folder) and os.path.isdir(common_folder_path):
        print(f"Subfolder '{common_keyword}' already exists within the base folder. Doing nothing.")
    else:
        os.makedirs(common_folder_path)
        print(f"Subfolder '{common_keyword}' created within the base folder.")

    return common_folder_path

# Convert pixel by pixel to qx by qy 2D SAXS plot

def convert_saxs_to_q(data, pixel_size, wavelength, detector_distance, beam_x, beam_y):
    # Define grid of pixel coordinates (assumes data is square)
    #x_pixels,y_pixels = data.shape
    x_pixels = 981
    y_pixels = 1043
    x_coords = np.arange(x_pixels) - beam_x
    y_coords = np.arange(y_pixels) - beam_y
    xx, yy = np.meshgrid(x_coords, y_coords)

    # Convert to q-space coordinates
    qx = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * xx / detector_distance)
    qy = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * yy / detector_distance)

    #print (len(xx))
    
    return qx, qy


    #print (qx)

# input varibales 

In [ ]:
# inputs
#poni files saxs and waxs
folder_path_poni_mask = '/Users/akmaurya/Library/CloudStorage/OneDrive-SLACNationalAcceleratorLaboratory/My Onedrive/Data_01/Methanolysis/Python/Data_Reduction/Convert_and_calibrate'
# Set the directory containing the raw and pdi files
raw_dir = '/Users/akmaurya/Library/CloudStorage/OneDrive-SLACNationalAcceleratorLaboratory/My Onedrive/Data_01/Methanolysis/Dec2023'
SAXS_poni_file = "Methanolysis_Dec2023_SAXS.poni"
WAXS_poni_file = "Methanolysis_Dec2023_WAXS_GUI.poni"


######################### to be change for every Run ########################
keyword="Run1*dx0*"

common_keyword = "Run1" 

##################################################################################

# Base folders
base_SAXS_folder = "OneD_integrated_SAXS_01_mask/Reduction"
base_SAXS_folder = os.path.join(raw_dir, base_SAXS_folder)

base_WAXS_folder = "OneD_integrated_WAXS_01/Reduction"
base_WAXS_folder = os.path.join(raw_dir, base_WAXS_folder)
# Create or update folders and get updated folder paths
SAXS_folder_name = create_update_subfolder(base_SAXS_folder, common_keyword)
WAXS_folder_name = create_update_subfolder(base_WAXS_folder, common_keyword)

print(SAXS_folder_name)

# Use SAXS_folder_name and WAXS_folder_name as needed in your code


# Read poni file for azimuthal integration

In [ ]:
poni_file_path = os.path.join(folder_path_poni_mask,SAXS_poni_file)
ai = pyFAI.load(poni_file_path)

print(ai)

#Define detector parameters for 2D q conversion
pixel_size = ai.get_pixel1() # in meters 
wavelength = ai.get_wavelength() #1.54e-10  # in meters
detector_distance = ai.get_dist()  # in meters
beam_x= 491.641
beam_y= 627.426 # beam center x-coordinate in pixels


# Set the directory containing the raw and pdi files and list the keyword

In [ ]:
import os
import fnmatch

def find_folders_with_keyword(folder_path, keyword, skip_folders=None):
    folder_names = []
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path) and fnmatch.fnmatch(item, keyword):
            if skip_folders is None or item not in skip_folders:
                folder_names.append(item)

    return folder_names


files = os.listdir(raw_dir)

# Input folder path and keyword
folder_path = raw_dir
skip_folders = ['OneD_integrated_SAXS_01_Norm']  # Replace with folder names to skip, or set to None to skip no folders

# Find folder names containing the keyword, skipping specified folders
folder_names_with_keyword = find_folders_with_keyword(folder_path, keyword, skip_folders)

# Extract "ctr" values from folder names
ctr_values = [int(name.split('_ctr')[-1]) for name in folder_names_with_keyword]

# Sort folder names based on "ctr" values
sorted_folder_names = [x for _, x in sorted(zip(ctr_values, folder_names_with_keyword))]

# Replace "dx0" with "*"
sorted_folder_names = [name.replace("dx0", "*") for name in sorted_folder_names]

keywords=(sorted_folder_names)

# Print the sorted folder names
print("keywords = ")
for sorted_name in sorted_folder_names:
    print(f"    '{sorted_name}',")

    
    

# Data reduction 

In [ ]:
import matplotlib.pyplot as plt
import re   

# Define the base folder name and create the folder

folder_path = SAXS_folder_name
print (folder_path)

filename_dx_1 = None 
filename_dx0 = None
filename_dx1 = None

for keyword in keywords:
    # Create a list of file paths that match the file types and current keyword
    raw_file_pattern = f'*{keyword}/SAXS/*.raw'
    print (raw_file_pattern)
    raw_file_paths = sorted(glob.glob(os.path.join(raw_dir, raw_file_pattern)))

    # Initialize the accumulators for the averaged image and pdi data
    avg_image = np.zeros((1043, 981))
    # Initialize variables to store averages
    avg_dx_minus_1 = np.zeros((1043, 981), dtype=np.int32)
    avg_dx_0 = np.zeros((1043, 981), dtype=np.int32)
    avg_dx_1 = np.zeros((1043, 981), dtype=np.int32)

    for i, raw_file_path in enumerate(raw_file_paths):
        # Read the raw file
        data = np.fromfile(raw_file_path, dtype=np.int32).reshape(1043, 981)
        avg_image += data

        file_name = os.path.splitext(os.path.basename(raw_file_path))[0]

        if "_dx-1_" in file_name:
            #print(file_name)
            filename_dx_1 = file_name
            filename_dx_1 = filename_dx_1[5:-10]
            #print(i)
            avg_dx_minus_1 += data
        elif "_dx0_" in file_name:
            avg_dx_0 += data
            #print(file_name)
            filename_dx0 = file_name
            filename_dx0 = filename_dx0[5:-10]
        elif "_dx1_" in file_name:
            avg_dx_1 += data
            filename_dx1 = file_name
            filename_dx1 = filename_dx1[5:-10]
            #print(file_name)
        else: 
            continue   
        
         # Calculate the final averages
    dx_1_size=len([f for f in raw_file_paths if "_dx-1_" in f])
    #print(dx_1_size)
    avg_dx_minus_1 = (avg_dx_minus_1 / dx_1_size)
    dx0_size = len([f for f in raw_file_paths if "_dx0_" in f])
    avg_dx_0 = (avg_dx_0 / dx0_size)
    dx1_size=len([f for f in raw_file_paths if "_dx1_" in f])
    avg_dx_1 = (avg_dx_1 /dx1_size)

    avg_image /= len(raw_file_paths)

    # Assuming raw_file_paths[-1] contains the file path
    file_name, file_extension = os.path.splitext(os.path.basename(raw_file_paths[-1]))

    # Remove the last 10 characters from the file name and the ".raw" extension
    file_name = file_name[5:-10]
    file_extension = ""

    avg_file_name = f"{file_name}{len(raw_file_paths)}average{file_extension}"
    #print(avg_file_name)
    #print (avg_file_name)

    # Create the modified filenames
    avg_dx_1_filename = f"{filename_dx_1}_{dx_1_size}average"
    #print (avg_dx_1_filename)
    avg_dx0_filename = f"{filename_dx0}_{dx0_size}average"
    avg_dx1_filename = f"{filename_dx1}_{dx1_size}average"
    
    
    #print (f"{keyword}_{num_raw_files}_average")
    set_pcolor_properties()
    fign, axsn = plt.subplots(figsize=(4,4))
    qx,qy=convert_saxs_to_q(avg_image, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    #im = axsn.pcolormesh(qx,qy,np.log10(avg_image), cmap='jet')
    im = axsn.pcolormesh(qx, qy, np.log10(avg_image), cmap='jet', vmin=-0.5, vmax=1.5)
    axsn.set_aspect('equal')
    cbar=fign.colorbar(im, ax=axsn, shrink=0.8, label='Intensity [a.u.]')


    #fign.colorbar(im, ax=axsn)
    axsn.set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axsn.set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axsn.set_title(f"{avg_file_name}",fontsize=14,y=1.05)
    # set tick levels
    axsn.set_xticks([-2, -1, 0, 1, 2])
    axsn.set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axsn.tick_params(axis='both', which='major', labelsize=20, width=1.5)

    #file_path = os.path.join(folder_path, f"_{avg_file_name}.png")
    #plt.savefig(file_path, dpi=300, bbox_inches='tight', format='png')
    plt.show()
    
 
    # Create a figure with a 2x2 grid of subplots
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Plot avg_image in the top-left subplot
    qx_avg, qy_avg = convert_saxs_to_q(avg_image, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_avg = axs[0, 0].pcolormesh(qx_avg, qy_avg, np.log10(avg_image), cmap='jet', vmin=-0.5, vmax=1.5)
    axs[0, 0].set_aspect('equal')
    cbar_avg = fig.colorbar(im_avg, ax=axs[0, 0], shrink=0.65, label='Intensity [a.u.]')
    axs[0, 0].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[0, 0].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[0, 0].set_title(f"{avg_file_name}", fontsize=14, y=1.05)
    axs[0, 0].set_xticks([-2, -1, 0, 1, 2])
    axs[0, 0].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[0, 0].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Plot avg_dx_minus_1 in the top-right subplot
    qx_minus_1, qy_minus_1 = convert_saxs_to_q(avg_dx_minus_1, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_minus_1 = axs[0, 1].pcolormesh(qx_minus_1, qy_minus_1, np.log10(avg_dx_minus_1), cmap='jet', vmin=-0.5, vmax=1.5)
    axs[0, 1].set_aspect('equal')
    cbar_minus_1 = fig.colorbar(im_minus_1, ax=axs[0, 1], shrink=0.65, label='Intensity [a.u.]')
    axs[0, 1].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[0, 1].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[0, 1].set_title("avg_dx_minus_1", fontsize=14, y=1.05)
    axs[0, 1].set_xticks([-2, -1, 0, 1, 2])
    axs[0, 1].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[0, 1].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Plot avg_dx_0 in the bottom-left subplot
    qx_0, qy_0 = convert_saxs_to_q(avg_dx_0, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_0 = axs[1, 0].pcolormesh(qx_0, qy_0, np.log10(avg_dx_0), cmap='jet', vmin=-0.5, vmax=1.5)
    axs[1, 0].set_aspect('equal')
    cbar_0 = fig.colorbar(im_0, ax=axs[1, 0], shrink=0.65, label='Intensity [a.u.]')
    axs[1, 0].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[1, 0].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[1, 0].set_title("avg_dx_0", fontsize=14, y=1.05)
    axs[1, 0].set_xticks([-2, -1, 0, 1, 2])
    axs[1, 0].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[1, 0].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Plot avg_dx_1 in the bottom-right subplot
    qx_1, qy_1 = convert_saxs_to_q(avg_dx_1, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_1 = axs[1, 1].pcolormesh(qx_1, qy_1, np.log10(avg_dx_1), cmap='jet', vmin=-0.5, vmax=1.5)
    axs[1, 1].set_aspect('equal')
    cbar_1 = fig.colorbar(im_1, ax=axs[1, 1], shrink=0.65, label='Intensity [a.u.]')
    axs[1, 1].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[1, 1].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[1, 1].set_title("avg_dx_1", fontsize=14, y=1.05)
    axs[1, 1].set_xticks([-2, -1, 0, 1, 2])
    axs[1, 1].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[1, 1].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Adjust the spacing between subplots
    plt.tight_layout()

    #file_path = os.path.join(folder_path, f"_{avg_file_name}_all.png")
    #plt.savefig(file_path, dpi=50, bbox_inches='tight', format='png')
    plt.show()


    
    
   # Create a figure with 1 row and 2 columns
    fig, axs = plt.subplots(2,1, figsize=(6, 12))

    for j, raw_file_path in enumerate(raw_file_paths):
        data = np.fromfile(raw_file_path, dtype=np.int32).reshape(1043, 981)
        file_name = os.path.splitext(os.path.basename(raw_file_path))[0]

        q, I, error = ai.integrate1d(data, 1000, error_model='poisson')#, filename=os.path.join(folder_path, f"{file_name}_SAXS.dat"))
        axs[0].loglog(q, I, label=file_name)
        #axs[0].set_xlabel("q (nm$^{-1}$)")
        axs[0].set_ylabel("Intensity (a.u.)")
        axs[0].set_xlim([0.05, 3])
        axs[0].legend(fontsize=12)
        axs[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=12)

    # Integrate and plot for avg_dx_minus_1
    q_dx_minus_1, I_dx_minus_1,error_dx_minus_1 = ai.integrate1d(avg_dx_minus_1, 1000, error_model='poisson', filename=os.path.join(folder_path, f"{avg_dx_1_filename}_dx_minus_1_SAXS.dat"))
    axs[1].loglog(q_dx_minus_1, I_dx_minus_1, '-g',label=f"{avg_dx_1_filename}_dx_minus_1")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)

    # Integrate and plot for avg_dx_0
    q_dx_0, I_dx_0, error_dx_0 = ai.integrate1d(avg_dx_0, 1000,  error_model='poisson',filename=os.path.join(folder_path, f"{avg_dx0_filename}_dx0_SAXS.dat"))
    axs[1].loglog(q_dx_0, I_dx_0, '-b',label=f"{avg_dx0_filename}_dx0")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)

    # Integrate and plot for avg_dx_1
    q_dx_1, I_dx_1, error_dx_1 = ai.integrate1d(avg_dx_1, 1000,  error_model='poisson', filename=os.path.join(folder_path, f"{avg_dx1_filename}_dx1_SAXS.dat"))
    axs[1].loglog(q_dx_1, I_dx_1, '-k',label=f"{avg_dx1_filename}_dx1")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)
    axs[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=12)
    
    q, I, error_avg = ai.integrate1d(avg_image, 1000,error_model='poisson', filename=os.path.join(folder_path, f"{avg_file_name}_all_SAXS.dat"))
    axs[1].loglog(q, I, '-r',label=f"{avg_file_name}_all")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)
    axs[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=12)

    

    plt.tight_layout()
    plt.show()

    filename_dx_1 = None 
    filename_dx0 = None
    filename_dx1 = None

# read parameters

In [ ]:
import pandas as pd
import os
import glob
import shutil
import pickle  # Import the pickle module

def process_and_store_selected_columns(raw_dir, keywords):
    # Create an empty dictionary to store the selected columns for each file
    selected_columns_dict = {}

    for keyword in keywords:
        # Create a list of file paths that match the file types and current keyword
        #raw_file_pattern = f'*{keyword}/SAXS/*.raw'
        pdi_file_pattern = f'*{keyword}/*_scan1.csv'
        #raw_file_paths = sorted(glob.glob(os.path.join(raw_dir, raw_file_pattern)))
        pdi_file_paths = sorted(glob.glob(os.path.join(raw_dir, pdi_file_pattern)))

        for pdi_file_path in pdi_file_paths:
            try:
                # Read the CSV file
                df = pd.read_csv(pdi_file_path)
                file_name = os.path.splitext(os.path.basename(pdi_file_path))[0]
                #print(f"Reading file: {file_name}")

                # Extract columns 4, 7, 9, and 15
                selected_columns = df.iloc[:, [2, 3, 6, 10, 29, 30]]

                # Store the selected columns in the dictionary with the file name as the key
                selected_columns_dict[file_name] = selected_columns

            except FileNotFoundError:
                print(f"File not found: {pdi_file_path}")
            except Exception as e:
                print(f"An error occurred: {e}")

    return selected_columns_dict

# Call the function and pass the raw_dir and keywords as arguments
#raw_dir = 'path_to_your_raw_files_directory'
#keywords = ['keyword1', 'keyword2', 'keyword3']  # Replace with your actual keywords
result_dict = process_and_store_selected_columns(raw_dir, keywords)

# Save the result_dict to a file using pickle
output_dir = folder_path  # Replace with your desired output directory
output_file = os.path.join(output_dir, 'result_dict.pkl')

try:
    with open(output_file, 'wb') as f:
        pickle.dump(result_dict, f)
    print(f"Result_dict saved to {output_file}")
except Exception as e:
    print(f"An error occurred while saving the result_dict: {e}")

# Now, result_dict contains the selected columns for each file
# You can access the data by using keys (file names) in the result_dict.

# To load the result_dict back from the file, you can use the following code:
with open(output_file, 'rb') as f:
     result_dict = pickle.load(f)
print(result_dict)

In [ ]:
import pandas as pd
import os
import glob
import shutil
import matplotlib.pyplot as plt

# Create lists to store averaged values
bstop_dx_avg = []
ctemp_dx_avg = []
timer_dx_avg = []
i0_dx_avg = []

bstop_avg_all = []
ctemp_avg_all = []
timer_avg_all = []
i0_avg_all = []

avg_file_name = []

# Call the function and pass the raw_dir and keywords as arguments
for keyword in keywords:
    
    i0_dx_avg_tmp=[]
    bstop_dx_avg_tmp=[]
    ctemp_dx_avg_tmp=[]
    timer_dx_avg_tmp=[]
    
    # Create a list of file paths that match the file types and current keyword
    #raw_file_pattern = f'*{keyword}/SAXS/*.raw'
    pdi_file_pattern = f'*{keyword}/*_scan1.csv'
    #raw_file_paths = sorted(glob.glob(os.path.join(raw_dir, raw_file_pattern)))
    pdi_file_paths = sorted(glob.glob(os.path.join(raw_dir, pdi_file_pattern)))
    
    for pdi_file_path in pdi_file_paths:
        file_name = os.path.splitext(os.path.basename(pdi_file_path))[0]
        #print (file_name)
        
        # Let's say you want to read individual columns from a file named "example_file"

        # Access the selected columns for the specified file
        selected_columns = result_dict.get(file_name)

        if selected_columns is not None:
            # Access individual columns by index or name
            i0 = selected_columns.iloc[:, 1]
            bstop = selected_columns.iloc[:, 2]  # Access the first column (index 2)
            ctemp = selected_columns.iloc[:, 4]  # Access the second column (index 4)
            timer = selected_columns.iloc[:, 5]  # Access the third column (index 5)
            #print (bstop)
            try:
                # Convert the columns to numeric values, skipping non-numeric data
                i0 = pd.to_numeric(i0, errors='coerce')
                bstop = pd.to_numeric(bstop, errors='coerce')
                ctemp = pd.to_numeric(ctemp, errors='coerce')
                timer = pd.to_numeric(timer, errors='coerce')
                
                #print (bstop)

                # Calculate the average values, excluding NaN values
                i0_average = i0.mean(skipna=True)
                bstop_average = bstop.mean(skipna=True)
                ctemp_average = ctemp.mean(skipna=True)
                timer_average = timer.mean(skipna=True)

                # Append the averages to the respective lists
                i0_dx_avg_tmp.append(i0_average)
                bstop_dx_avg_tmp.append(bstop_average)
                ctemp_dx_avg_tmp.append(ctemp_average)
                timer_dx_avg_tmp.append(timer_average)
                
                i0_dx_avg.append(i0_average)
                bstop_dx_avg.append(bstop_average)
                ctemp_dx_avg.append(ctemp_average)
                timer_dx_avg.append(timer_average)
                
                
                #print (f'{file_name}={ctemp_average}')
            except ValueError as e:
                print(f"Error processing file '{file_name}': {str(e)}")
        else:
            print(f"File '{file_name}' not found in the selected_columns_dict.")
    #print (file_name)
    
    avg_file_name_tmp = f'{file_name}_all_average'
    avg_file_name.append(avg_file_name_tmp)
    #print (avg_file_name)
    # Calculate the average values, excluding NaN values
    
    i0_average_tmp = sum(i0_dx_avg_tmp) / len(i0_dx_avg_tmp)
    bstop_average_tmp = sum(bstop_dx_avg_tmp) / len(bstop_dx_avg_tmp)
    ctemp_average_tmp = sum(ctemp_dx_avg_tmp) / len(ctemp_dx_avg_tmp)
    timer_average_tmp = sum(timer_dx_avg_tmp) / len(timer_dx_avg_tmp)
    
    # Append the averages to the respective lists
    i0_avg_all.append(i0_average_tmp)
    bstop_avg_all.append(bstop_average_tmp)
    ctemp_avg_all.append(ctemp_average_tmp)
    timer_avg_all.append(timer_average_tmp)
    
    #print (ctemp_average_tmp)
    
#print (ctemp_dx_avg)   
    
# Create a figure and axis for plotting
#print (f'{ctemp_averages}')

# Initialize the result list with the first value as 0
time_durations = [0]
duration_csv_avg_all = [0]

# Subtract the first value from the rest and append to the result list
for i in range(1, len(timer_dx_avg)):
    duration = (timer_dx_avg[i] - timer_dx_avg[0])/3600
    time_durations.append(duration)


# Subtract the first value from the rest and append to the result list
for i in range(1, len(timer_avg_all)):
    duration_avg = (timer_avg_all[i] - timer_avg_all[0])/3600
    duration_csv_avg_all.append(duration_avg)


#print(f"time={duration_avg_all}_ctemp_all={ctemp_avg_all}\n")
# Now time_durations contains the time durations
#print(time_durations)


# Get the first item from keywords and use the first 2 characters as the file name prefix
first_keyword = keywords[0]
file_name_prefix = first_keyword[:4]

# Plot the parameter values against Timer in the same plot
fig3, axs3 = plt.subplots(figsize=(10, 6))
axs3.set_facecolor('white')


#axis_plot_prop(axs3)

# Plot with marker size 10 and transparent legend
#axs3.plot(time_duration, i0_values, '-o', markersize=10, label='i0')    
#axs3.plot(time_durations, bstop_averages, 'o', markersize=10, label='bstop')
#axs3.plot(time_duration, temp_values, '-<', markersize=10, label='TEMP')
axs3.plot(time_durations, ctemp_dx_avg,'ok', markersize=10, label='CTEMP')

axs3.plot(duration_csv_avg_all, ctemp_avg_all,'*r', markersize=10, label='CTEMP_avg')


axs3.set_xlabel('Time Duration [Hrs]')
axs3.set_ylabel('Temperature [°C]')
#axs3.set_title('Parameter values vs Timer')

#axs3.legend(framealpha=0.5)  # Set legend transparency

plt.tight_layout()


plt.show()



import os
import glob
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

# Function to parse the date and time string
def parse_datetime(datetime_str):
    return datetime.strptime(datetime_str, "%a %b %d %H:%M:%S %Y")

# Function to calculate duration in seconds
def calculate_duration(start_time, end_time):
    return (end_time - start_time).total_seconds()

# Function to convert seconds to hours
def seconds_to_hours(seconds):
    return timedelta(seconds=seconds).total_seconds() / 3600.0

# Initialize offset as None
offset = None
durations = []  # List to store durations in seconds
real_times = []  # List to store real times
file_names = []  # List to store file names
durations_avg_all = []

# Iterate through files in the folder
for keyword in keywords:
    # Create a list of file paths that match the file types and current keyword
    spec_file_pattern = f'*{keyword}/{keyword}'
    file_paths = sorted(glob.glob(os.path.join(raw_dir, spec_file_pattern)))
    durations_tmp = []
    for file_path in file_paths: 
        #file_name = os.path.splitext(os.path.basename(pdi_file_path))[0]
        file_name = os.path.basename(file_path) 
        #print(file_name)
        with open(file_path, 'r') as file:
            lines = file.readlines()
            start_time = None
            end_time = None

            for line in lines:
                if line.startswith("#D"):
                    if start_time is None:
                        if offset is None:
                            offset = parse_datetime(line[3:].strip())
                        start_time = parse_datetime(line[3:].strip())
                    end_time = parse_datetime(line[3:].strip())

            if start_time and end_time:
                duration_seconds = calculate_duration(offset, end_time)
                duration_hours = seconds_to_hours(duration_seconds)
                durations_tmp.append(duration_hours)
                durations.append(duration_hours)# Store duration in hours
                real_times.append(start_time)  # Store real time
                file_name = os.path.basename(file_path)
                file_names.append(file_name)
            else:
                print(f"Start and end times not found in {file_path}")
    durations_avg_tmp = sum(durations_tmp)/len(durations_tmp)
    durations_avg_all.append(durations_avg_tmp)
#print(durations_avg_all)
# Create a DataFrame to store the data
data = {
    "Filename": file_names,
    "Real Time": real_times,
    "duration_realtime_hr":durations,
    "time_duration_CSV": time_durations,
    "ctemp": ctemp_dx_avg,
    "i0": i0_dx_avg,
    "bstop": bstop_dx_avg
    
}



# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Get the first item from keywords and use the first 2 characters as the file name prefix
first_keyword = keywords[0]
file_name_prefix = first_keyword[:4]

# Save the DataFrame to an Excel file
output_folder = folder_path  # Replace with your desired folder path
output_file_name = f'{file_name_prefix}_dx_time_temp_i0_bstop.xlsx'  # Replace with your desired file name
output_file_path = os.path.join(output_folder, output_file_name)
df.to_excel(output_file_path, index=False)
#print (f 'dx average parameters saved to {output_file_path}')
############################ Average
data_avg = {
    "Filename": avg_file_name,
    "duration_realtime_hr":durations_avg_all,
    "ctemp": ctemp_avg_all,
    "i0": i0_avg_all,
    "bstop": bstop_avg_all
    
}


# Convert the data to a pandas DataFrame
df = pd.DataFrame(data_avg)

# Get the first item from keywords and use the first 2 characters as the file name prefix
first_keyword = keywords[0]
file_name_prefix = first_keyword[:4]

# Save the DataFrame to an Excel file
output_folder = folder_path  # Replace with your desired folder path
output_file_name = f'{file_name_prefix}_avg_all_time_temp_i0_bstop.xlsx'  # Replace with your desired file name
output_file_path_avg = os.path.join(output_folder, output_file_name)
df.to_excel(output_file_path_avg, index=False)


# Plotting (optional)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(file_names, durations)
plt.ylabel("Duration (hours)")
plt.title("Duration vs. Filename")
plt.xlabel("#Filename")
plt.xticks([])  # Remove x-axis ticks
plt.grid(True)
plt.tight_layout()
plt.show()

print(f'dx average parameters saved to {output_file_path}')
print(f'all dx averaged parameters saved to {output_file_path_avg}')


# WAXS

In [ ]:
# import matplotlib.pyplot as plt

def set_pcolor_properties():
    """
    Sets the best figure properties for pcolormap plot
    """
    plt.rcParams["figure.figsize"] = (6, 6)  # Set the figure size to 8 inches by 6 inches
    plt.rcParams["font.size"] = 20  # Set the font size to 12 points
    plt.rcParams["axes.linewidth"] = 1.5  # Set the linewidth of the axes to 1.5
    #plt.rcParams["axes.grid"] = True  # Show the grid
    #plt.rcParams["grid.alpha"] = 0.5  # Set the transparency of the grid to 0.5
    #plt.rcParams["grid.linewidth"] = 0.5  # Set the linewidth of the grid to 0.5
    plt.rcParams["image.cmap"] = "viridis"  # Set the colormap to viridis
    plt.rcParams["image.interpolation"] = "nearest"  # Set the interpolation to nearest

def convert_saxs_to_q(data, pixel_size, wavelength, detector_distance, beam_x, beam_y):
    # Define grid of pixel coordinates (assumes data is square)
    #x_pixels,y_pixels = data.shape
    x_pixels = 487
    y_pixels = 195
    x_coords = np.arange(x_pixels) - beam_x
    y_coords = np.arange(y_pixels) - beam_y
    xx, yy = np.meshgrid(x_coords, y_coords)

    # Convert to q-space coordinates
    qx = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * xx / detector_distance)
    qy = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * yy / detector_distance)

    print (len(xx))
    
    return qx, qy


    #print (qx) 

In [ ]:

# Load the poni file and extract relevant parameters
poni_file_path = os.path.join(folder_path_poni_mask, WAXS_poni_file)
ai = pyFAI.load(poni_file_path)

print(ai)



#Define detector parameters for 2D q conversion
pixel_size = ai.get_pixel1() # in meters 
wavelength = ai.get_wavelength() #1.54e-10  # in meters
detector_distance = ai.get_dist()  # in meters
beam_x= 644.072
beam_y= 106.130 # beam center x-coordinate in pixels


In [ ]:
   
# Define the base folder name and create the folder

folder_path = WAXS_folder_name

import matplotlib.pyplot as plt
import re

filename_dx_1 = None 
filename_dx0 = None
filename_dx1 = None

for keyword in keywords:
    # Create a list of file paths that match the file types and current keyword
    raw_file_pattern = f'*{keyword}/WAXS/*.raw'
    raw_file_paths = sorted(glob.glob(os.path.join(raw_dir, raw_file_pattern)))

    # Initialize the accumulators for the averaged image and pdi data
    avg_image = np.zeros((195, 487))

    # Initialize variables to store averages
    avg_dx_minus_1 = np.zeros((195, 487), dtype=np.int32)
    avg_dx_0 = np.zeros((195, 487), dtype=np.int32)
    avg_dx_1 = np.zeros((195, 487), dtype=np.int32)

    for i, raw_file_path in enumerate(raw_file_paths):
        

        # Read the raw file
        data = np.fromfile(raw_file_path, dtype=np.int32).reshape(195, 487)
        avg_image += data

        file_name = os.path.splitext(os.path.basename(raw_file_path))[0]

        if "_dx-1_" in file_name:
            #print(file_name)
            filename_dx_1 = file_name
            filename_dx_1 = filename_dx_1[10:-10]
            #print(i)
            avg_dx_minus_1 += data
        elif "_dx0_" in file_name:
            avg_dx_0 += data
            #print(file_name)
            filename_dx0 = file_name
            filename_dx0 = filename_dx0[10:-10]
        elif "_dx1_" in file_name:
            avg_dx_1 += data
            filename_dx1 = file_name
            filename_dx1 = filename_dx1[10:-10]
            #print(file_name)
        

    # Calculate the final averages
    dx_1_size=len([f for f in raw_file_paths if "_dx-1_" in f])
    #print(dx_1_size)
    avg_dx_minus_1 = (avg_dx_minus_1 / dx_1_size)
    dx0_size = len([f for f in raw_file_paths if "_dx0_" in f])
    avg_dx_0 = (avg_dx_0 / dx0_size)
    dx1_size=len([f for f in raw_file_paths if "_dx1_" in f])
    avg_dx_1 = (avg_dx_1 /dx1_size)

    avg_image /= len(raw_file_paths)


    # Assuming raw_file_paths[-1] contains the file path
    file_name, file_extension = os.path.splitext(os.path.basename(raw_file_paths[-1]))
    
    print (file_name)

    # Remove the last 10 characters from the file name and the ".raw" extension
    file_name = file_name[10:-10]
    file_extension = ""

    avg_file_name = f"{file_name}{len(raw_file_paths)}average{file_extension}"
    print(avg_file_name)

    #print (avg_file_name)
      
    # Remove 5 characters from the beginning and 10 characters from the end
    #filename_dx_1 = filename_dx_1[5:-10]
    #filename_dx0 = filename_dx0[5:-10]
    #filename_dx1 = filename_dx1[5:-10]

    # Create the modified filenames
    avg_dx_1_filename = f"{filename_dx_1}_{dx_1_size}average"
    #print (avg_dx_1_filename)
    avg_dx0_filename = f"{filename_dx0}_{dx0_size}average"
    avg_dx1_filename = f"{filename_dx1}_{dx1_size}average"

    
    #print (f"{keyword}_{num_raw_files}_average")
    set_pcolor_properties()
    fign, axsn = plt.subplots(figsize=(6,6))
    qx,qy=convert_saxs_to_q(avg_image, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    #im = axsn.pcolormesh(qx,qy,np.log10(avg_image), cmap='jet')
    im = axsn.pcolormesh(-qx, -qy, np.log10(avg_image), cmap='jet')#, vmin=0, vmax=3)
    axsn.set_aspect('equal')
    cbar=fign.colorbar(im, ax=axsn, shrink=0.4, label='Intensity [a.u.]')


    #fign.colorbar(im, ax=axsn)
    axsn.set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axsn.set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axsn.set_title(f"{avg_file_name}",fontsize=10,y=1.3)
    # set tick levels
    #axsn.set_xticks([-2, -1, 0, 1, 2])
    #axsn.set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axsn.tick_params(axis='both', which='major', labelsize=20, width=1.5)

    file_path = os.path.join(folder_path, f"_{avg_file_name}.png")
    plt.savefig(file_path, dpi=300, bbox_inches='tight', format='png')
    plt.show()
    
    # Create a figure with a 2x2 grid of subplots
    fig, axs = plt.subplots(2, 2, figsize=(12, 7))

    # Plot avg_image in the top-left subplot
    qx_avg, qy_avg = convert_saxs_to_q(avg_image, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_avg = axs[0, 0].pcolormesh(-qx_avg, -qy_avg, np.log10(avg_image), cmap='jet')#, vmin=0, vmax=5)
    axs[0, 0].set_aspect('equal')
    cbar_avg = fig.colorbar(im_avg, ax=axs[0, 0], shrink=0.8, label='Intensity [a.u.]')
    axs[0, 0].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[0, 0].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[0, 0].set_title(f"{avg_file_name}", fontsize=14, y=1.3)
    #axs[0, 0].set_xticks([-2, -1, 0, 1, 2])
    #axs[0, 0].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[0, 0].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Plot avg_dx_minus_1 in the top-right subplot
    qx_minus_1, qy_minus_1 = convert_saxs_to_q(avg_dx_minus_1, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_minus_1 = axs[0, 1].pcolormesh(-qx_minus_1, -qy_minus_1, np.log10(avg_dx_minus_1), cmap='jet')#, vmin=0, vmax=5)
    axs[0, 1].set_aspect('equal')
    cbar_minus_1 = fig.colorbar(im_minus_1, ax=axs[0, 1], shrink=0.8, label='Intensity [a.u.]')
    axs[0, 1].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[0, 1].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[0, 1].set_title("avg_dx_minus_1", fontsize=14, y=1.3)
    #axs[0, 1].set_xticks([-2, -1, 0, 1, 2])
    #axs[0, 1].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[0, 1].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Plot avg_dx_0 in the bottom-left subplot
    qx_0, qy_0 = convert_saxs_to_q(avg_dx_0, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_0 = axs[1, 0].pcolormesh(-qx_0, -qy_0, np.log10(avg_dx_0), cmap='jet')#, vmin=0, vmax=5)
    axs[1, 0].set_aspect('equal')
    cbar_0 = fig.colorbar(im_0, ax=axs[1, 0], shrink=0.8, label='Intensity [a.u.]')
    axs[1, 0].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[1, 0].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[1, 0].set_title("avg_dx_0", fontsize=14, y=1.3)
    #axs[1, 0].set_xticks([-2, -1, 0, 1, 2])
    #axs[1, 0].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[1, 0].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Plot avg_dx_1 in the bottom-right subplot
    qx_1, qy_1 = convert_saxs_to_q(avg_dx_1, pixel_size, wavelength, detector_distance, beam_x, beam_y)
    im_1 = axs[1, 1].pcolormesh(-qx_1, -qy_1, np.log10(avg_dx_1), cmap='jet')
    axs[1, 1].set_aspect('equal')
    cbar_1 = fig.colorbar(im_1, ax=axs[1, 1], shrink=0.8, label='Intensity [a.u.]')
    axs[1, 1].set_xlabel(r"$q_x$ [nm$^{-1}$]")
    axs[1, 1].set_ylabel(r"$q_y$ [nm$^{-1}$]")
    axs[1, 1].set_title("avg_dx_1", fontsize=14, y=1.3)
    #axs[1, 1].set_xticks([-2, -1, 0, 1, 2])
    #axs[1, 1].set_yticks([-2.4, -1.6, -0.8, 0.0, 0.8, 1.6])
    axs[1, 1].tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Adjust the spacing between subplots
    plt.tight_layout()

    #file_path = os.path.join(folder_path, f"_{avg_file_name}_all.png")
    #plt.savefig(file_path, dpi=50, bbox_inches='tight', format='png')
    plt.show()

    
    
    import matplotlib.pyplot as plt
    import numpy as np
    import os

    # Create a figure with 1 row and 2 columns
    fig, axs = plt.subplots(2,1, figsize=(6, 12))

    for j, raw_file_path in enumerate(raw_file_paths):
        data = np.fromfile(raw_file_path, dtype=np.int32).reshape(195, 487)
        file_name = os.path.splitext(os.path.basename(raw_file_path))[0]

        q, I, error = ai.integrate1d(data, 1000, error_model='poisson')#, filename=os.path.join(folder_path, f"{file_name}_SAXS.dat"))
        axs[0].plot(q, I, label=file_name)
        #axs[0].set_xlabel("q (nm$^{-1}$)")
        axs[0].set_ylabel("Intensity (a.u.)")
        #axs[0].set_xlim([0.05, 3])
        axs[0].legend(fontsize=12)
        axs[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=12)

    # Integrate and plot for avg_dx_minus_1
    q_dx_minus_1, I_dx_minus_1,error_dx_minus_1 = ai.integrate1d(avg_dx_minus_1, 1000,error_model='poisson', filename=os.path.join(folder_path, f"{avg_dx_1_filename}_dx_minus_1_WAXS.dat"))
    axs[1].plot(q_dx_minus_1, I_dx_minus_1, '-g',label=f"{avg_dx_1_filename}_dx_minus_1")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    #axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)

    # Integrate and plot for avg_dx_0
    q_dx_0, I_dx_0, error_dx_0 = ai.integrate1d(avg_dx_0, 1000,error_model='poisson', filename=os.path.join(folder_path, f"{avg_dx0_filename}_dx0_WAXS.dat"))
    axs[1].plot(q_dx_0, I_dx_0, '-b',label=f"{avg_dx0_filename}_dx0")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    #axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)

    # Integrate and plot for avg_dx_1
    q_dx_1, I_dx_1,error_dx_1 = ai.integrate1d(avg_dx_1, 1000,error_model='poisson', filename=os.path.join(folder_path, f"{avg_dx1_filename}_dx1_WAXS.dat"))
    axs[1].plot(q_dx_1, I_dx_1, '-k',label=f"{avg_dx1_filename}_dx1")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    #axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)
    axs[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=12)
    
    q, I, error_avg = ai.integrate1d(avg_image, 1000,error_model='poisson', filename=os.path.join(folder_path, f"{avg_file_name}_all_WAXS.dat"))
    axs[1].plot(q, I, '-r',label=f"{avg_file_name}_all")
    axs[1].set_xlabel("q (nm$^{-1}$)")
    axs[1].set_ylabel("Intensity (a.u.)")
    #axs[1].set_xlim([0.05, 3])
    axs[1].legend(fontsize=12)
    axs[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=12)

    plt.tight_layout()
    plt.show()
    
    filename_dx_1 = None 
    filename_dx0 = None
    filename_dx1 = None